# Working with parquet files

## Objective

+ In this assignment, we will use the data downloaded with the module `data_manager` to create features.

(11 pts total)

## Prerequisites

+ This notebook assumes that price data is available to you in the environment variable `PRICE_DATA`. If you have not done so, then execute the notebook `01_materials/labs/2_data_engineering.ipynb` to create this data set.


+ Load the environment variables using dotenv. (1 pt)

In [7]:
# Write your code below.
%reload_ext dotenv
%dotenv ../../01_materials/labs/.env


In [8]:
import dask.dataframe as dd

+ Load the environment variable `PRICE_DATA`.
+ Use [glob](https://docs.python.org/3/library/glob.html) to find the path of all parquet files in the directory `PRICE_DATA`.

(1pt)

In [10]:
import os
import pandas as pd
from glob import glob

# Write your code below.
PRICE_DATA = os.getenv("PRICE_DATA")
parquet_files = glob(os.path.join(PRICE_DATA, "*/*/*.parquet"))
parquet_files


['../../05_src/data/prices\\A\\A_2000.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2001.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2002.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2003.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2004.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2005.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2006.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2007.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2008.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2009.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2010.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2011.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2012.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2013.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2014.parquet\\part.0.parquet',
 '../../05_src/data/prices\\A\\A_2015.pa

For each ticker and using Dask, do the following:

+ Add lags for variables Close and Adj_Close.
+ Add returns based on Adjusted Close:
    
    - `returns`: (Adj Close / Adj Close_lag) - 1

+ Add the following range: 

    - `hi_lo_range`: this is the day's High minus Low.

+ Assign the result to `dd_feat`.

(4 pt)

In [11]:
# Write your code below.

dd_px = dd.read_parquet(parquet_files).set_index("ticker")

dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(Close_lag = x['Close'].shift(1),
                       Adj_Close_lag = x['Adj Close'].shift(1))
).assign(
    returns = lambda x: x['Adj Close']/x['Adj_Close_lag'] - 1
).assign(
    hi_lo_range = lambda x: x['High'] - x['Low']
))

c:\Users\pm\miniconda3\envs\dsi_participant\lib\site-packages\dask\dataframe\core.py:5517: UserWarning: New index has same name as existing, this is a no-op.
  warnings.warn(
C:\Users\pm\AppData\Local\Temp\ipykernel_18788\2169396382.py:5: UserWarning: `meta` is not specified, inferred from partial data. Please provide `meta` if the result is unexpected.
  Before: .apply(func)
  After:  .apply(func, meta={'x': 'f8', 'y': 'f8'}) for dataframe result
  or:     .apply(func, meta=('x', 'f8'))            for series result
  dd_feat = (dd_px.groupby('ticker', group_keys=False).apply(


+ Convert the Dask data frame to a pandas data frame. 
+ Add a rolling average return calculation with a window of 10 days.
+ *Tip*: Consider using `.rolling(10).mean()`.

(3 pt)

In [12]:
# Write your code below.

import pandas as pd
import numpy as np

df = dd_feat.compute()
df = (df.groupby('ticker', group_keys=False).apply(
    lambda x: x.assign(rolling_avg_return = x['returns'].rolling(10).mean())
))
df

,Date,Open,High,Low,Close,Adj Close,Volume,sector,subsector,year,Close_lag,Adj_Close_lag,returns,hi_lo_range,rolling_avg_return
ticker,,,,,,,,,,,,,,,
HUM,2001-01-02,14.500000,15.250000,14.312500,14.937500,13.337626,933100,Health Care,Managed Health Care,2001,NaN,NaN,NaN,0.937500,NaN
HUM,2001-01-03,14.687500,15.625000,14.250000,14.437500,12.891180,1077200,Health Care,Managed Health Care,2001,14.937500,13.337626,-0.033473,1.375000,NaN
HUM,2001-01-04,14.375000,14.437500,12.250000,13.187500,11.775059,1217600,Health Care,Managed Health Care,2001,14.437500,12.891180,-0.086580,2.187500,NaN
HUM,2001-01-05,13.000000,14.250000,12.750000,13.625000,12.165703,1606100,Health Care,Managed Health Care,2001,13.187500,11.775059,0.033176,1.500000,NaN
HUM,2001-01-08,13.625000,14.500000,13.312500,13.562500,12.109896,873700,Health Care,Managed Health Care,2001,13.625000,12.165703,-0.004587,1.187500,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
JNPR,2019-12-24,24.510000,24.510000,24.270000,24.290001,21.141632,714900,Information Technology,Communications Equipment,2019,24.469999,21.298304,-0.007356,0.240000,0.002319
JNPR,2019-12-26,24.290001,24.530001,24.290001,24.490000,21.315706,1141000,Information Technology,Communications Equipment,2019,24.290001,21.141632,0.008234,0.240000,0.002005
JNPR,2019-12-27,24.570000,24.610001,24.389999,24.410000,21.246071,3030300,Information Technology,Communications Equipment,2019,24.490000,21.315706,-0.003267,0.220001,0.000262


Please comment:

+ Was it necessary to convert to pandas to calculate the moving average return?
No , Its not necessary to convert and we can do this with dask
+ Would it have been better to do it in Dask? Why?
When we working with Large Data , Its better to work with Dask.  Dask is designed for handling large datasets that don’t fit into memory, while pandas works best with smaller datasets that fit into memory, and it can be better in parallelism and memory efficiency

(1 pt)

## Criteria

The [rubric](./assignment_1_rubric_clean.xlsx) contains the criteria for grading.

## Submission Information

🚨 **Please review our [Assignment Submission Guide](https://github.com/UofT-DSI/onboarding/blob/main/onboarding_documents/submissions.md)** 🚨 for detailed instructions on how to format, branch, and submit your work. Following these guidelines is crucial for your submissions to be evaluated correctly.

### Submission Parameters:
* Submission Due Date: `HH:MM AM/PM - DD/MM/YYYY`
* The branch name for your repo should be: `assignment-1`
* What to submit for this assignment:
    * This Jupyter Notebook (assignment_1.ipynb) should be populated and should be the only change in your pull request.
* What the pull request link should look like for this assignment: `https://github.com/<your_github_username>/production/pull/<pr_id>`
    * Open a private window in your browser. Copy and paste the link to your pull request into the address bar. Make sure you can see your pull request properly. This helps the technical facilitator and learning support staff review your submission easily.

Checklist:
- [ ] Created a branch with the correct naming convention.
- [ ] Ensured that the repository is public.
- [ ] Reviewed the PR description guidelines and adhered to them.
- [ ] Verify that the link is accessible in a private browser window.

If you encounter any difficulties or have questions, please don't hesitate to reach out to our team via our Slack at `#cohort-3-help`. Our Technical Facilitators and Learning Support staff are here to help you navigate any challenges.